<a href="https://colab.research.google.com/github/Aki-alphon/Aki-alphon/blob/main/food_delivery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [75]:
import pandas as pd
import sqlite3


In [76]:
orders = pd.read_csv("orders.csv")

In [77]:
users = pd.read_json("users.json")

In [78]:
conn = sqlite3.connect("restaurants.db")
cursor = conn.cursor()

with open("restaurants.sql", "r") as file:
  sql_script = file.read()

cursor.executescript(sql_script)
conn.commit()
conn.close()


In [73]:
!rm restaurants.db

In [79]:
conn = sqlite3.connect("restaurants.db")

restaurants = pd.read_sql_query(
    "SELECT * FROM restaurants",
    conn
)

conn.close()

restaurants.head()


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [80]:
restaurant_master = (
    orders[["restaurant_id", "restaurant_name"]]
    .drop_duplicates()
)


In [81]:
restaurants = restaurant_master.merge(
    restaurants,
    on="restaurant_id",
    how="left"
)


In [82]:
final_df = orders.merge(users, on="user_id", how="left" )
final_df = final_df.merge(restaurants, on="restaurant_id", how="left")

In [83]:
final_df.columns


Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
       'restaurant_name', 'name', 'city', 'membership', 'restaurant_name_x',
       'restaurant_name_y', 'cuisine', 'rating'],
      dtype='object')

In [84]:
final_df.rename(columns={
    "city_x": "user_city",
    "city_y": "restaurant_city"
}, inplace=True)


In [85]:
orders.shape[0], final_df.shape[0]


(10000, 10000)

In [120]:
final_df.isnull().sum()


,0
order_id,0
user_id,0
restaurant_id,0
order_date,0
total_amount,0
restaurant_name,0
name,0
city,0
membership,0
restaurant_name_x,0


In [169]:
final_df = final_df[[
    "order_id",
    "order_date",
    "user_id",
    "name",
    "city",
    "membership",
    "restaurant_id",
    "restaurant_name",
    "cuisine",
    "rating",
    "total_amount"
]]


In [170]:
final_df.head()


,order_id,order_date,user_id,name,city,membership,restaurant_id,restaurant_name,cuisine,rating,total_amount
0,1,2023-02-18,2508,User_2508,Hyderabad,Regular,450,New Foods Chinese,Mexican,3.2,842.97
1,2,2023-01-18,2693,User_2693,Pune,Regular,309,Ruchi Curry House Multicuisine,Indian,4.5,546.68
2,3,2023-07-15,2084,User_2084,Chennai,Gold,107,Spice Kitchen Punjabi,Mexican,4.0,163.93
3,4,2023-10-04,319,User_319,Bangalore,Gold,224,Darbar Kitchen Non-Veg,Chinese,4.8,1155.97
4,5,2023-12-25,1064,User_1064,Pune,Regular,293,Royal Eatery South Indian,Italian,3.0,1321.91


In [143]:
final_df.columns


Index(['order_id', 'order_date', 'user_id', 'name', 'city', 'membership',
       'restaurant_id', 'restaurant_name', 'restaurant_name', 'cuisine',
       'rating', 'total_amount'],
      dtype='object')

In [144]:
final_df = final_df.loc[:, ~final_df.columns.duplicated()]


In [145]:
final_df.columns


Index(['order_id', 'order_date', 'user_id', 'name', 'city', 'membership',
       'restaurant_id', 'restaurant_name', 'cuisine', 'rating',
       'total_amount'],
      dtype='object')

1.Which city has the highest total revenue (total_amount) from Gold members?  

In [126]:
gold_orders = final_df[final_df["membership"]=="Gold"]

In [127]:
city_revenue = (
    gold_orders
    .groupby("city")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)


In [128]:
city_revenue

,total_amount
city,
Chennai,1080909.79
Pune,1003012.32
Bangalore,994702.59
Hyderabad,896740.19


2.Which cuisine has the highest average order value across all orders?

In [129]:
cuisine_aov = (
    final_df
    .groupby("cuisine")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)

In [130]:
cuisine_aov

,total_amount
cuisine,
Mexican,808.021344
Italian,799.448578
Indian,798.466011
Chinese,798.389020


3.How many distinct users placed orders worth more than ₹1000 in total (sum of all their orders)?

In [93]:
total_spend_by_user = (
    final_df
    .groupby("user_id")["total_amount"]
    .sum()
)

In [94]:
high_value = total_spend_by_user[total_spend_by_user>1000]

In [95]:
high_value_count = high_value.count()
high_value_count



np.int64(2544)

4.Which restaurant rating range generated the highest total revenue?

In [96]:
import numpy as np

conditions = [
    (final_df["rating"] >= 3.0) & (final_df["rating"] <= 3.5),
    (final_df["rating"] > 3.5) & (final_df["rating"] <= 4.0),
    (final_df["rating"] > 4.0) & (final_df["rating"] <= 4.5),
    (final_df["rating"] > 4.5) & (final_df["rating"] <= 5.0),
]

labels = [
    "3.0 – 3.5",
    "3.6 – 4.0",
    "4.1 – 4.5",
    "4.6 – 5.0"
]

final_df["rating_range"] = np.select(conditions, labels, default="Other")

In [97]:
rating_revenue = (
    final_df.groupby("rating_range")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

rating_revenue

,total_amount
rating_range,
4.6 – 5.0,2197030.75
3.0 – 3.5,2136772.70
4.1 – 4.5,1960326.26
3.6 – 4.0,1717494.41


In [98]:
rating_revenue.idxmax()


'4.6 – 5.0'

5.Among Gold members, which city has the highest average order value?

In [52]:
city_avg_order = (
    gold_orders
    .groupby("city")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)

city_avg_order

,total_amount
city,
Chennai,808.459080
Hyderabad,806.421034
Bangalore,793.223756
Pune,781.162243


6.Which cuisine has the lowest number of distinct restaurants but still contributes significant revenue?

In [131]:
restaurant_count = (
    final_df
    .groupby("cuisine")["restaurant_id"]
    .nunique()
    .reset_index(name="restaurant_count")
)

In [132]:
revenue_by_cuisine =(
    final_df
    .groupby("cuisine")["total_amount"]
    .sum()
    .reset_index(name="total_revenue")
)

In [133]:
cuisine_summary = restaurant_count.merge(
    revenue_by_cuisine,
    on="cuisine"
)

cuisine_summary.sort_values(
    ["restaurant_count","total_revenue"],
    ascending=[True, False]
)

,cuisine,restaurant_count,total_revenue
0,Chinese,120,1930504.65
2,Italian,126,2024203.80
1,Indian,126,1971412.58
3,Mexican,128,2085503.09


7.What percentage of total orders were placed by Gold members? (Rounded to nearest integer)

In [134]:
total_orders = final_df["order_id"].count()
gold_orders = final_df[final_df["membership"] == "Gold"]["order_id"].count()
percentage = round((gold_orders / total_orders) * 100)
percentage


50

8.Which restaurant has the highest average order value but less than 20 total orders?

In [147]:
options = [
    "Grand Cafe Punjabi",
    "Grand Restaurant South Indian",
    "Ruchi Mess Multicuisine",
    "Ruchi Foods Chinese"
]

filtered_df = final_df[
    final_df["restaurant_name"].isin(options)
]


In [148]:
stats = (
    filtered_df
    .groupby("restaurant_name")
    .agg(
        total_orders=("order_id", "count"),
        avg_order_value=("total_amount", "mean")
    )
    .reset_index()
)


In [149]:
stats = stats[stats["total_orders"] < 20]
stats.sort_values("avg_order_value", ascending=False)


,restaurant_name,total_orders,avg_order_value
2,Ruchi Foods Chinese,19,686.603158


8.Which combination contributes the highest revenue?

In [151]:
combo_revenue = (
    final_df
    .groupby(["membership", "cuisine"])["total_amount"]
    .sum()
    .reset_index()
)

options = [
    ("Gold", "Indian"),
    ("Gold", "Italian"),
    ("Regular", "Indian"),
    ("Regular", "Chinese")
]

filtered = combo_revenue[
    combo_revenue.apply(
        lambda x: (x["membership"], x["cuisine"]) in options,
        axis=1
    )
]

filtered.sort_values("total_amount", ascending=False)




,membership,cuisine,total_amount
2,Gold,Italian,1005779.05
5,Regular,Indian,992100.27
1,Gold,Indian,979312.31
4,Regular,Chinese,952790.91


In [152]:
filtered_df = final_df[
    ((final_df["membership"] == "Gold") &
     (final_df["cuisine"].isin(["Indian", "Italian"]))) |
    ((final_df["membership"] == "Regular") &
     (final_df["cuisine"].isin(["Indian", "Chinese"])))
]


In [153]:
result = (
    filtered_df
    .groupby(["membership", "cuisine"])["total_amount"]
    .sum()
    .reset_index()
    .sort_values("total_amount", ascending=False)
)

result


,membership,cuisine,total_amount
1,Gold,Italian,1005779.05
3,Regular,Indian,992100.27
0,Gold,Indian,979312.31
2,Regular,Chinese,952790.91


In [154]:
result["total_amount"].sum()


np.float64(3929982.54)

9.During which quarter of the year is the total revenue highest?


In [155]:
final_df["order_date"] = pd.to_datetime(final_df["order_date"])


/tmp/ipython-input-486593379.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df["order_date"] = pd.to_datetime(final_df["order_date"])


In [156]:
final_df["quarter"] = final_df["order_date"].dt.quarter


In [157]:
quarter_revenue = (
    final_df
    .groupby("quarter")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

quarter_revenue


,total_amount
quarter,
3,2037385.10
4,2018263.66
1,2010626.64
2,1945348.72


In [159]:
gold_orders = final_df[final_df["membership"] == "Gold"]["order_id"].count()
gold_orders


np.int64(4987)

In [160]:
round(
    final_df[final_df["city"] == "Hyderabad"]["total_amount"].sum()
)


1889367

In [161]:
final_df["user_id"].nunique()


2883

In [162]:
round(
    final_df[final_df["membership"] == "Gold"]["total_amount"].mean(),
    2
)


np.float64(797.15)

In [163]:
final_df[final_df["rating"] >= 4.5]["order_id"].count()


np.int64(3374)

In [166]:
top_city = (
    final_df[final_df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .idxmax()
)

top_city



'Chennai'

In [167]:
final_df[
    (final_df["membership"] == "Gold") &
    (final_df["city"] == top_city)
]["order_id"].count()


np.int64(1337)

In [168]:
final_df.shape[0]


10000